# Big data with Dask

The following code is part of the youtube videos below,

1. [Download building footprint dataset from Google](https://youtu.be/R0ElIJS4C70?si=P5DI7Rz67EtKrMqd)
2. [Working with big data in panadas](https://youtu.be/7kBL83my5I0?si=ta1k0KT-no-Pzfi5)

In [ ]:
import pandas as pd 
import numpy as np 
import geopandas as gpd 

## Dataset
There are two input dataset;
1. 20 GB csv file
This is the dataset downloaded from [Google's open buildings](https://sites.research.google/open-buildings/#download).
2. AOI shapefile
AOI shapefile is the created on [geojson.io](https://geojson.io) website. It is the small area near Kathmandu.

In [ ]:
df = pd.read_csv(r"C:\Users\tek.kshetri\Downloads\30f_buildings.csv\30f_buildings.csv")
df.head()

In [ ]:
df['geometry'] = gpd.GeoSeries.from_wkt(df['geometry'])
gdf = gpd.GeoDataFrame(df, geometry='geometry')
gdf.head()

In [ ]:
gdf.to_file(r"C:\Users\tek.kshetri\Downloads\687_buildings.shp")

In [ ]:
import dask.dataframe as dd

ddf = dd.read_csv(r"C:\Users\tek.kshetri\Downloads\39f_buildings.csv\39f_buildings.csv")
ddf.head()

In [ ]:
import dask_geopandas
# ddf['geometry'] = ddf.set_geometry(dask_geopandas.from_wkt(ddf['geometry']))

In [ ]:
# add ddf to the geopandas dataframe using dask geopandas 
gddf = dask_geopandas.from_dask_dataframe(ddf, 
                                          geometry=ddf['geometry']
                                          .map_partitions(gpd.GeoSeries
                                                          .from_wkt, meta=gpd.GeoSeries([])))

gddf = gddf.reset_index()

In [ ]:
gddf.crs = 'epsg:4326'

In [ ]:
aoi = gpd.read_file(r"C:\Users\tek.kshetri\Downloads\download\layers\POLYGON.shp")
aoi.head()

In [ ]:
aoi.crs = 'epsg:4326'

In [ ]:
gddf = gddf.to_crs(aoi.crs)

In [ ]:
## clip the buildings to the aoi
clipped = dask_geopandas.clip(gddf, aoi)

In [ ]:
clipped.head()

In [ ]:
clipped.to_file(r"C:\Users\tek.kshetri\Downloads\test_buildings.shp")

In [ ]:
clipped = clipped.compute()
clipped.to_file(r"C:\Users\tek.kshetri\Downloads\test_buildings.shp")